In [14]:
%matplotlib notebook
import sigpy.plot as pl
from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
import numpy as np
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import scipy.io
from torch.autograd import Variable
# import UFNet
import os
# import bart
from torch import optim
import torch_utils as flare
import resnet
# import bart
import sigpy as sp
import matplotlib.pyplot as plt
# from skimage.measure import compare_ssim
import unet.unet_model as UN
import os
from modl_model import MoDL_model,UnrolledNetwork,CG_adj_3D
from pytorch3dunet.unet3d.model import UNet3D
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="5"
# import 
# device = torch.device('cpu')
device = torch.device('cuda:0')
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Defined function

In [2]:
def load_slab(folder,case,center,slices):
    start = center - int(slices/2)
    for j in range(slices):
        if j == 0:
            knee_ksp = np.load("/home/kewang/cube_knee/%s_ksp_slices/%d_%d.npy"%(folder,case,start))[None,...]
            knee_im = np.load("/home/kewang/cube_knee/%s_img_slices/%d_%d.npy"%(folder,case,start))[None,...]
            knee_mps = np.load("/home/kewang/cube_knee/%s_mps_slices/%d_%d.npy"%(folder,case,start))[None,...]
            knee_masks = np.load("/home/kewang/cube_knee/%s_mask_slices/%d_%d.npy"%(folder,case,start))[None,None,...]
        else:
            knee_ksp = np.concatenate((knee_ksp,np.load("/home/kewang/cube_knee/%s_ksp_slices/%d_%d.npy"%(folder,case,start+j))[None,...]))
            knee_im = np.concatenate((knee_im,np.load("/home/kewang/cube_knee/%s_img_slices/%d_%d.npy"%(folder,case,start+j))[None,...]))
            knee_mps = np.concatenate((knee_mps,np.load("/home/kewang/cube_knee/%s_mps_slices/%d_%d.npy"%(folder,case,start+j))[None,...]))
            knee_masks = np.concatenate((knee_masks,np.load("/home/kewang/cube_knee/%s_mask_slices/%d_%d.npy"%(folder,case,start+j))[None,None,...]))
    return  knee_ksp,knee_im,knee_mps,knee_masks
def bart_valid(knee_ksp,knee_mps,knee_masks):
    ksp_under = (knee_ksp*knee_masks).transpose((1,2,0))[None,...]
    ksp_mps = knee_mps.transpose((1,2,0))[None,...]
    return bart.bart(1,"pics -l1 -r 0.02 -S",ksp_under,ksp_mps)
def CG_adj(ksp,mps,mask):
    SenseModel = flare.SenseModel(mps,mask) 
    adj = SenseModel.adjoint(ksp)
    return SenseModel,adj
def CG_adj_3D(ksp,mps,mask):
    SenseModel = flare.SenseModel_3D(mps,mask) 
    adj = SenseModel.adjoint(ksp)
    return SenseModel,adj
def CG_MoDL_3D(ksp,mps,mask,lam = 0):
    SenseModel = flare.SenseModel_3D(mps,mask)    
    adj = SenseModel.adjoint(ksp)
    CG_alg = flare.ConjGrad(Aop_fun=SenseModel.normal,b=adj,verbose=False,l2lam=lam)
    return CG_alg.forward(adj)
def CG_MoDL(ksp,mps,mask,lam = 0):
    SenseModel = flare.SenseModel(mps,mask)    
    adj = SenseModel.adjoint(ksp)
    CG_alg = flare.ConjGrad(Aop_fun=SenseModel.normal,b=adj,verbose=False,l2lam=lam)
    return CG_alg.forward(adj)
def load_data(folder,dir_data,dev,flag=0,mask=False):
    knee_ksp = np.load("/mikRAID/frank/data/cube_knees/%s_ksp_slices/%s"%(folder,dir_data))
    knee_im = np.load("/mikRAID/frank/data/cube_knees/%s_img_slices/%s"%(folder,dir_data))[None,...]
    knee_mps = np.load("/mikRAID/frank/data/cube_knees/%s_mps_slices/%s"%(folder,dir_data))
    knee_masks = np.load("/mikRAID/frank/data/cube_knees/%s_mask_slices/%s"%(folder,dir_data))[None,...]
    if flag:
        knee_masks = mask
    knee_ksp = bart.bart(1,"fftmod 6",knee_ksp)
    knee_mps = bart.bart(1,"fftmod 6",knee_mps)
    ksp_under = (knee_ksp*knee_masks)[None,...]
    knee_mps = knee_mps[None,...]
    im_torch = flare.np2torch(knee_im)
    ksp_torch = flare.np2torch(ksp_under)
    mps_torch = flare.np2torch(knee_mps)
    mask_torch = flare.np2torch(abs(knee_masks),data_complex=False)
    return ksp_torch.to(dev),mps_torch.to(dev),mask_torch.to(dev),im_torch.to(dev)
class MoDL_3D(nn.Module):
    def __init__(self,M=None,A = None,lam_l2 = 0,unroll = 3,cg_max = 10):
        super(MoDL_3D, self).__init__()
        self.Model = M
        self.A = A
        self.lam2 = lam_l2
        self.urnum = unroll
        self.cg = cg_max

    def forward(self, adj):
        out = adj
#         print(out.shape)
        for i in range(self.urnum):
            print(i)
            out = self.Model(out).squeeze(0).permute(1,2,3,0)
#             out1 = out
#             print(adj.shape)
#             print(out.shape)
#             print(out.shape)
#             print(adj.squeeze(0).permute(1,2,3,0).shape)
            rhs = adj.squeeze(0).permute(1,2,3,0) + self.lam2 * out
            CG_alg = flare.ConjGrad(Aop_fun=self.A.normal,b=rhs,verbose=False,l2lam=self.lam2,max_iter=self.cg)
            out = CG_alg.forward(rhs).permute(3,0,1,2).unsqueeze(0)
#             print(out.shape)
        return out

In [3]:
def nmrse(x,gt):
    return np.linalg.norm(x-gt)/np.linalg.norm(gt)
def psnr(x,gt):
    mse = np.linalg.norm(x-gt)**2/(x.shape[0]*x.shape[1])
#     print(mse)
    return 20 * np.log10(abs(gt).max()/np.sqrt(mse)) 

# Perform Testing

In [4]:
# Hyper parameters of the training
folder = "test"
case = 20
slices = 21
metadata = {'num_unrolls': 4, 'lamb': 5e-2,'cg':8}
Loss = nn.L1Loss()


In [5]:
modl_network = MoDL_model(metadata,device=device)
modl_network.network.load_state_dict((torch.load("/home/kewang/3D_DIR_results/checkpoints_0520/cp_unroll_sl6_4_cg_8_MELD_epochs_23.pth", map_location=device)))

<All keys matched successfully>

In [7]:
with torch.no_grad():
    knee_ksp,knee_im,knee_mps,knee_masks = load_slab(folder,case,180,slices)
    im_torch = flare.np2torch(knee_im).to(device).permute(3,0,1,2).unsqueeze(0)
    ksp_under = (knee_ksp*knee_masks)
    ksp_torch = flare.np2torch(ksp_under).to(device)
    mps_torch = flare.np2torch(knee_mps).to(device)
    mask_torch = flare.np2torch(abs(knee_masks),data_complex=False).to(device)
    Sense,adj = CG_adj_3D(ksp_torch,mps_torch,mask_torch)
    modl_network.initialize(Sense,adj)
    out = modl_network.forward(adj)

In [21]:
with torch.no_grad():
    for center in range(10,305,21):
        print(center)
        knee_ksp,knee_im,knee_mps,knee_masks = load_slab(folder,case,center,slices)
        im_torch = flare.np2torch(knee_im).to(device).permute(3,0,1,2).unsqueeze(0)
        ksp_under = (knee_ksp*knee_masks)
        ksp_torch = flare.np2torch(ksp_under).to(device)
        mps_torch = flare.np2torch(knee_mps).to(device)
        mask_torch = flare.np2torch(abs(knee_masks),data_complex=False).to(device)
        Sense,adj = CG_adj_3D(ksp_torch,mps_torch,mask_torch)
#         adj_input = adj.permute(3,0,1,2).unsqueeze(0)
        modl_network.initialize(Sense,adj)
        out = modl_network.forward(adj)
        if center == 10:
            im_all_3ddir = flare.torch2np(out[0,...].permute(1,2,3,0))
        else:
            im_all_3ddir = np.concatenate((im_all_3ddir,flare.torch2np(out[0,...].permute(1,2,3,0))))
    center = 309
    knee_ksp,knee_im,knee_mps,knee_masks = load_slab(folder,case,center,slices)
    im_torch = flare.np2torch(knee_im).to(device).permute(3,0,1,2).unsqueeze(0)
    ksp_under = (knee_ksp*knee_masks)
    ksp_torch = flare.np2torch(ksp_under).to(device)
    mps_torch = flare.np2torch(knee_mps).to(device)
    mask_torch = flare.np2torch(abs(knee_masks),data_complex=False).to(device)
    Sense,adj = CG_adj_3D(ksp_torch,mps_torch,mask_torch)
    modl_network.initialize(Sense,adj)
    out = modl_network.forward(adj)
    im_temp = flare.torch2np(out[0,...].permute(1,2,3,0))[-5:,...]
    im_all_3ddir = np.concatenate((im_all_3ddir,im_temp))

10
31
52
73
94
115
136
157
178
199
220
241
262
283
304
